# Example of using the orm classes

Imports

In [1]:
from core.core import Project
from core.orm import BoreholeOrm, PositionOrm, Base
from striplog import Lexicon, Striplog
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

In [2]:
! rm ./tmp/test_orm_db.db

Create the db engine and the database structure

In [3]:
engine = create_engine('sqlite:///tmp/test_orm_db.db', echo=True)

In [4]:
Base.metadata.create_all(engine)

2020-12-18 19:09:27,613 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-12-18 19:09:27,615 INFO sqlalchemy.engine.base.Engine ()
2020-12-18 19:09:27,621 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-12-18 19:09:27,621 INFO sqlalchemy.engine.base.Engine ()
2020-12-18 19:09:27,623 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Boreholes")
2020-12-18 19:09:27,624 INFO sqlalchemy.engine.base.Engine ()
2020-12-18 19:09:27,628 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("Boreholes")
2020-12-18 19:09:27,629 INFO sqlalchemy.engine.base.Engine ()
2020-12-18 19:09:27,630 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Positions")
2020-12-18 19:09:27,630 INFO sqlalchemy.engine.base.Engine ()
2020-12-18 19:09:27,633 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("Positions")
2020-12-18 19:09:27,636 INFO sqlalchemy.engine.base.Engine ()
2020-12-18

Create the objects from las files

In [5]:
lexicon = Lexicon.default()

In [6]:
borehole_dict = {'F01':'../data/test.las', 'F02':'../data/test.las'} # boreholes to insert into the db

In [7]:
int_id = 0
bh_id = 0
pos_id = 0
boreholes = []
components = []
comp_id = 0
component_dict={}
pos_dict = {}
x = [0., 20.]
y = [0., 20.]
for bh, filename in borehole_dict.items():
    interval_number = 0
    boreholes.append(BoreholeOrm(id=bh))
    with open(filename, 'r') as las3:
        strip = Striplog.from_las3(las3.read(), lexicon)
    for c in strip.components:
        if c not in component_dict.keys():
            component_dict.update({c:comp_id})
            comp_id += 1
    d ={}
    for interval in strip:
        top = PositionOrm(id=pos_id, upper=interval.top.upper, middle=interval.top.middle,  
                          lower=interval.top.lower, x=x[bh_id], y=y[bh_id])
        base = PositionOrm(id=pos_id+1, upper=interval.base.upper, middle=interval.base.middle,  
                           lower=interval.base.lower, x=x[bh_id], y=y[bh_id])
        d.update({int_id:{'description':interval.description, 'interval_number' : interval_number, 
                          'top': top, 'base': base 
                         }})
        interval_number+=1
        int_id += 1
        pos_id += 2

    print(d)
    boreholes[bh_id].intervals_values = d
    #boreholes[bh_id].components_values = c
    bh_id += 1 
components = {v:k for k,v in component_dict.items()}

{0: {'description': 'Anhydrite', 'interval_number': 0, 'top': <core.orm.PositionOrm object at 0x7fdb736c9550>, 'base': <core.orm.PositionOrm object at 0x7fdb736c9908>}, 1: {'description': 'Sandstone, grey, vf-f', 'interval_number': 1, 'top': <core.orm.PositionOrm object at 0x7fdb736c9320>, 'base': <core.orm.PositionOrm object at 0x7fdb736c9dd8>}, 2: {'description': 'Anhydrite', 'interval_number': 2, 'top': <core.orm.PositionOrm object at 0x7fdb736c96d8>, 'base': <core.orm.PositionOrm object at 0x7fdb736c9588>}, 3: {'description': 'Dolomite', 'interval_number': 3, 'top': <core.orm.PositionOrm object at 0x7fdb736c9a20>, 'base': <core.orm.PositionOrm object at 0x7fdb736c9978>}, 4: {'description': 'Anhydrite', 'interval_number': 4, 'top': <core.orm.PositionOrm object at 0x7fdb736c9b38>, 'base': <core.orm.PositionOrm object at 0x7fdb736c99b0>}, 5: {'description': 'Sandstone, grey, vf-f', 'interval_number': 5, 'top': <core.orm.PositionOrm object at 0x7fdb736c9d30>, 'base': <core.orm.Position

Create the session

In [8]:
Session = sessionmaker(bind=engine)
session = Session()

Create the project

In [9]:
p = Project(session)
p.add_components(components)

2020-12-18 19:09:27,997 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-18 19:09:28,003 INFO sqlalchemy.engine.base.Engine SELECT "Boreholes".id AS "Boreholes_id" 
FROM "Boreholes"
2020-12-18 19:09:28,007 INFO sqlalchemy.engine.base.Engine ()
2020-12-18 19:09:28,018 INFO sqlalchemy.engine.base.Engine INSERT INTO "Components" (id, description) VALUES (?, ?)
2020-12-18 19:09:28,020 INFO sqlalchemy.engine.base.Engine ((0, 'Siltstone, grey'), (1, 'Anhydrite'), (2, 'Sandstone, vf-f, grey'), (3, 'Dolomite'), (4, 'Siltstone, red'), (5, 'Limestone'))
2020-12-18 19:09:28,026 INFO sqlalchemy.engine.base.Engine COMMIT
2020-12-18 19:09:28,038 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-18 19:09:28,045 INFO sqlalchemy.engine.base.Engine SELECT "Boreholes".id AS "Boreholes_id" 
FROM "Boreholes"
2020-12-18 19:09:28,050 INFO sqlalchemy.engine.base.Engine ()


Add boreholes into the database

In [10]:
for bh in boreholes:
    p.add_borehole(bh)

2020-12-18 19:09:28,262 INFO sqlalchemy.engine.base.Engine INSERT INTO "Positions" (id, upper, middle, lower, x, y) VALUES (?, ?, ?, ?, ?, ?)
2020-12-18 19:09:28,263 INFO sqlalchemy.engine.base.Engine ((0, 14.3, 14.3, 14.3, 0.0, 0.0), (1, 51.24117647058824, 51.24117647058824, 51.24117647058824, 0.0, 0.0), (2, 51.24117647058824, 51.24117647058824, 51.24117647058824, 0.0, 0.0), (3, 54.81764705882354, 54.81764705882354, 54.81764705882354, 0.0, 0.0), (4, 54.81764705882354, 54.81764705882354, 54.81764705882354, 0.0, 0.0), (5, 56.55882352941177, 56.55882352941177, 56.55882352941177, 0.0, 0.0), (6, 56.55882352941177, 56.55882352941177, 56.55882352941177, 0.0, 0.0), (7, 58.86470588235295, 58.86470588235295, 58.86470588235295, 0.0, 0.0)  ... displaying 10 of 50 total bound parameter sets ...  (48, 131.47647058823532, 131.47647058823532, 131.47647058823532, 0.0, 0.0), (49, 135.9, 135.9, 135.9, 0.0, 0.0))
2020-12-18 19:09:28,269 INFO sqlalchemy.engine.base.Engine INSERT INTO "Boreholes" (id) VALU

2020-12-18 19:09:28,473 INFO sqlalchemy.engine.base.Engine (14,)
2020-12-18 19:09:28,483 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2020-12-18 19:09:28,485 INFO sqlalchemy.engine.base.Engine (15,)
2020-12-18 19:09:28,489 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2020-12-18 19:09:28,496 INFO sqlalchemy.engine.base.Engine (16,)
2020-12-18 19:09:28,503 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positi

2020-12-18 19:09:28,700 INFO sqlalchemy.engine.base.Engine (35,)
2020-12-18 19:09:28,708 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2020-12-18 19:09:28,715 INFO sqlalchemy.engine.base.Engine (36,)
2020-12-18 19:09:28,727 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2020-12-18 19:09:28,729 INFO sqlalchemy.engine.base.Engine (37,)
2020-12-18 19:09:28,733 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positi

2020-12-18 19:09:29,073 INFO sqlalchemy.engine.base.Engine (50,)
2020-12-18 19:09:29,077 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2020-12-18 19:09:29,077 INFO sqlalchemy.engine.base.Engine (51,)
2020-12-18 19:09:29,082 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2020-12-18 19:09:29,083 INFO sqlalchemy.engine.base.Engine (52,)
2020-12-18 19:09:29,085 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positi

2020-12-18 19:09:29,279 INFO sqlalchemy.engine.base.Engine (71,)
2020-12-18 19:09:29,288 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2020-12-18 19:09:29,289 INFO sqlalchemy.engine.base.Engine (72,)
2020-12-18 19:09:29,299 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2020-12-18 19:09:29,302 INFO sqlalchemy.engine.base.Engine (73,)
2020-12-18 19:09:29,306 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positi

2020-12-18 19:09:29,481 INFO sqlalchemy.engine.base.Engine (92,)
2020-12-18 19:09:29,486 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2020-12-18 19:09:29,487 INFO sqlalchemy.engine.base.Engine (93,)
2020-12-18 19:09:29,491 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2020-12-18 19:09:29,495 INFO sqlalchemy.engine.base.Engine (94,)
2020-12-18 19:09:29,501 INFO sqlalchemy.engine.base.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positi

In [11]:
p.boreholes[0].id

'F01'

In [12]:
p.commit()

2020-12-18 19:09:29,793 INFO sqlalchemy.engine.base.Engine COMMIT


In [13]:
p.boreholes[0].intervals[0].description

2020-12-18 19:09:29,832 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-18 19:09:29,836 INFO sqlalchemy.engine.base.Engine SELECT "Boreholes".id AS "Boreholes_id" 
FROM "Boreholes" 
WHERE "Boreholes".id = ?
2020-12-18 19:09:29,840 INFO sqlalchemy.engine.base.Engine ('F01',)
2020-12-18 19:09:29,846 INFO sqlalchemy.engine.base.Engine SELECT "Intervals".id AS "Intervals_id", "Intervals".borehole AS "Intervals_borehole", "Intervals".interval_number AS "Intervals_interval_number", "Intervals".description AS "Intervals_description", "Intervals".top_id AS "Intervals_top_id", "Intervals".base_id AS "Intervals_base_id" 
FROM "Intervals" 
WHERE ? = "Intervals".borehole
2020-12-18 19:09:29,848 INFO sqlalchemy.engine.base.Engine ('F01',)


'Anhydrite'

In [14]:
session.close()

2020-12-18 19:09:29,868 INFO sqlalchemy.engine.base.Engine ROLLBACK
